# 동빈나 님의 Transformer를 구현 코드를 보면서 그대로 작성한 코드이다.

In [ ]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# 데이터 전처리.
## spaCy 라이브러리는 문장의 tokenization, tagging등의 전처리를 위한 라이브러리.
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

2023-05-11 14:11:31.091636: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-11 14:11:32.469410: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-11 14:11:33.766097: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-11 14:11:33.766538: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [ ]:
import spacy

spacy_en = spacy.load('en_core_web_sm') # 영어 tokenization
spacy_de = spacy.load('de_core_news_sm') # 독일어 tokenization

동빈나 코드에 없던 부분

In [ ]:
spacy_en.max_length

## END

1000000

In [ ]:
# 간단히 토큰화기능 써보기

tokenized = spacy_en.tokenizer("I am a graduate student.")

for i, token in enumerate(tokenized):
  print(f"인덱스 {i}: {token.text}")

인덱스 0: I
인덱스 1: am
인덱스 2: a
인덱스 3: graduate
인덱스 4: student
인덱스 5: .


In [ ]:
# 영어 및 독일어의 토큰화 함수 정의

#독일어 문장을 토큰화 하는 함수
def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]

#영어 문장을 토큰화 하는 함수
def tokenize_en(text):
  return [token.text for token in spacy_en.tokenizer(text)]

In [ ]:
# torchtext의 field 라이브러리를 이용해 데이터셋에 대한 구체적인 전처리 내용을 명시한다.
# Seq2Seq 모델과는 다르게, batch_first 속성 값을 True로 설정한다.
# 번역 목표: 
  # 소스(SRC): 독일어
  # 목표(TRG): 영어

from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)
TRG = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)

In [ ]:
# 대표적인 영어-독일어 번역 데이터셋인 Multi30k를 불러온다.

from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de", ".en"), fields = (SRC, TRG))

### Field:
- Field는 앞으로 어떤 전처리를 할 것인지를 정의한다. 이미지 처리에서 torchvision의 transforms같은 함수로 보면 좋을 듯 하다.
  - tokenize: 어떤 토큰화 함수를 사용할 것인지 지정 (Default는 string.split)
  - lower: 영어 데이터를 전무 소문자화한다. (Default는 False)
  - batch_first: 미니배치 차원을 맨 앞으로 할 것인지 (Defalut는 False)
  

In [ ]:
print(f"학습 데이터셋(training dataset) 크기: {len(train_dataset.examples)}개")
print(f"평가 데이터셋(validation dataset) 크기: {len(valid_dataset.examples)}개")
print(f"테스트 데이터셋(testing dataset) 크기: {len(test_dataset.examples)}개")

학습 데이터셋(training dataset) 크기: 29000개
평가 데이터셋(validation dataset) 크기: 1014개
테스트 데이터셋(testing dataset) 크기: 1000개


In [ ]:
# 학습 데이터 중 하나를 선택해 출력
print(vars(train_dataset.examples[30])['src'])
print(vars(train_dataset.examples[30])['trg'])

['ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.']
['a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.']


In [ ]:
# Field 객체의 build vocab 메서드를 이용해 영어와 독어의 단어 사전을 생성한다.
  # 최소 2번 이상 등장한 단어만을 선택한다.

SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 7853
len(TRG): 5893


min_freq: 몇 번이상 등장한 단어를 임베딩, 토큰의 대상으로써 vocabulary에 추가할 것인가?

In [ ]:
print(TRG.vocab.stoi["abcabc"])
print(TRG.vocab.stoi[TRG.pad_token])
print(TRG.vocab.stoi["<sos>"])
print(TRG.vocab.stoi["<eos>"])
print(TRG.vocab.stoi["hello"]) # 동빈나 코드에서는 4112로 매핑됬는데 여기선 0이 나왔다.
print(TRG.vocab.stoi["world"])


0
1
2
3
4112
1752


vocab.stoi는 현재 단어 집합의 단어와 맵핑된 고유한 정수를 출력할 수 있다.

하나의 배치에 포함된 문장들이 가지는 단어의 개수가 유사하도록 만들면 좋다고 한다.
- 이를 위해 BucketIterator를 사용한다 한다.

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 2

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_dataset, valid_dataset, test_dataset),
    batch_size = BATCH_SIZE,
    device=device)

In [ ]:
for i,batch in enumerate(train_iterator):
  src = batch.src
  trg = batch.trg

  print(f"첫 번째 배치 크기; {src.shape}")

  for i in range(src.shape[1]): # 이건 첫번째 배치의 첫번째 데이터에 대해서, 문장의 각각 토큰 값을 출력하는거
    print(f"인덱스 {i}: {src[0][i].item()}")

  break

첫 번째 배치 크기; torch.Size([2, 13])
인덱스 0: 2
인덱스 1: 8
인덱스 2: 67
인덱스 3: 12
인덱스 4: 6
인덱스 5: 102
인덱스 6: 728
인덱스 7: 68
인덱스 8: 1146
인덱스 9: 20
인덱스 10: 123
인덱스 11: 4
인덱스 12: 3


Multi Head Attention 아키텍처
- 어텐션(attention)은 세 가지 요소를 입력으로 받음
  - Query, Key, Value

- Hyperparameter는 다음과 같다.
  - Hidden_dim, n_heads, dropout_ratio

In [ ]:
import torch.nn as nn

class MultiHeadAttentionLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
    super().__init__()

    assert hidden_dim % n_heads == 0

    self.hidden_dim = hidden_dim # 임베딩 차원 
    self.n_heads = n_heads # 헤드 개수 
    self.head_dim = hidden_dim // n_heads # Multi head에서 헤드 여러개 나옴, 거기서 각각 헤드의 임베딩 차원

    # 맨 처음에 토큰 자체를 임베딩하는건 없나? 토큰 -> 단어를 실수 벡터로 임베딩 -> query, key, value로 나뉘는 임베딩
    self.fc_q = nn.Linear(hidden_dim, hidden_dim) # Query에 적용될 FC 레이어라고 함.
    self.fc_k = nn.Linear(hidden_dim, hidden_dim)
    self.fc_v = nn.Linear(hidden_dim, hidden_dim)

    self.fc_o = nn.Linear(hidden_dim, hidden_dim) 

    self.dropout = nn.Dropout(dropout_ratio)

    self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

  def forward(self, query, key, value, mask = None):

    batch_size = query.shape[0]

    # query, key, value의 shape는 다음과 같다.
    # query: [batch_size, query_len, hidden_dim]
    # key: [batch_size, key_len, hidden_dim]
    # value: [batch_size, value_len, hidden_dim]

    Q = self.fc_q(query) # 아직 입력은 token의 임베딩 상태이고 query로 임베딩을 거친게 아닌가? 입력의 이름만 query?
    K = self.fc_k(key)
    V = self.fc_k(value)


    Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
    V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

    # permute 수행 전
    # Q: [batch_size, n_heads, query_len, head_dim] # query len은 하나의 문장에 있는 단어의 개수, 문장 길이라고 보면 될듯.
    # K: [batch_size, n_heads, key_len, head_dim]
    # V: [batch_size, n_heads, value_len, head_dim]    

    # permute 수행 후
    # Q: [batch_size, n_heads, query_len, head_dim] -> permute는 의미는 그대로 두면서 shape 위치만 바꾸는거.
    # K: [batch_size, n_heads, key_len, head_dim]
    # V: [batch_size, n_heads, value_len, head_dim]


    # Attention 계산
    energy = torch.matmul(Q, K.permute(0,1,3,2)) / self.scale # 여기서 K reshape한건 matrix multiplicate 할 때 한쪽은 Transpose하는 것을 의미한거.

    # energy: [batch_size, n_heads, query_len, key_len]

    print("START the attention shape: ", energy.shape)
    print(energy)

  
    if mask is not None:
      energy = energy.masked_fill(mask==0, -1e10) # 반드시 어텐션은 마스크를 계산하고 나서 진행해야한다.

      print("After attention masking: ", energy.shape)
      print(energy)

      print("")

    # [batch_size, n_heads, query_len, key_len]에서 하나의 query에 대해 key 각각을 가지고 softmax 적용하는 것. 하나의 쿼리에 대해 모든 key에 해당하는 단어들의 softmax를 보겠다는 것.
    attention = torch.softmax(energy, dim=-1)

    # attention: [batch_size, n_heads, query_len, key_len]


    # 여기에서 Scaled Dot-Product Attention을 계산
    x = torch.matmul(attention, V)

    # x: [batch_size, n_heads, query_len, head_dim]

    # x: [batch_size, n_heads, query_len, head_dim]

    x = x.permute(0, 2, 1, 3).contiguous()

    # x: [batch_size, query_len, n_heads, head_dim]

    x = x.view(batch_size, -1, self.hidden_dim)

    # x: [batch_size, query_len, hidden_dim]

    x = self.fc_o(x)

    # x: [batch_size, query_len, hidden_dim]

    return x, attention

위의 궁금한 것 테스트

## 1. reshape의 형태

In [ ]:
# Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
# 이것이 어떻게 작동하는지 보자. 결국 앞에건 다 때고, hidden_dim → n_heads X head_dim 로 변하는 것을 보면 된다.
# 이걸 하는 이유는 reshape를 했을 때, shape크기는 우리가 원해도 순서가 우리가 원하는 대로 가는지는 파악이 힘들다.

_test = torch.tensor([0,1,2,3,4,5,6,7,8,9,10,11])
_test.view(-1,3)

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]])

d

즉, hidden_dim이 `[0,1,2,3,4,5,6,7,8,9,10,11]` 이렇게 있으면,
이것이 [n_heads, head_dim]꼴로  
`[[ 0,  1,  2],  
        [ 3,  4,  5],  
        [ 6,  7,  8],  
        [ 9, 10, 11]]`

이렇게 되는 것. 이건 우리가 형태가 맞음


## 2. Pad mask에서 왜 mask==0 을 하는지

mask == 0 에서 mask는 그냥 인자가 아님. 이것도 우리가 정의해줘야 하는 torch tensor 형태의 tensor이다.

https://thought-process-ing.tistory.com/79 에서 보면 mask도 따로 정의해서 하고있다. mask는 input과 동일한 shape를 하고, mask = [[1,2,3],[0,2,0]] 이라 할 때, 만약 mask에서 0인 위치에 원본 input을 1e-10으로 하고싶다면 mask==0, 1e-10으로 하는거. pad가 1인거랑은 상관없다. mask의 shape와 mask의 어느 위치에 0이 있는지가 중요한거 -> src_mask를 어떻게 정의하는지를 봐야되는거.


Transformer class의 밑에 부분이 핵심이다.

In [ ]:
def make_src_mask(self, src):

  # src: [batch_size, src_len]

  src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

  # src_mask: [batch_size, 1, 1, src_len]

  return src_mask

이게 masking하는건데, src != self.src_pad_idx -> src의 형태는 일단  
[[I, am, a, boy, <pad>,<pad>,<pad>,<pad>],  
[I, am, kind, of, human, do, not, forget]]  
의 정수화 형태인  
->  
[[19, 2, 3, 4, 1,1,1,1],  
[19, 2, 5, 6, 7,8,9,11]]   
형태임.

여기서 self.src_pad_idx == 1 이므로 (src != self.src_pad_idx)는 (src != 1)라는 의미고 이를 적용하면  
[[True, True, True, True, False,False,False,False],  
[True, True, True, True, True,True,True,True]]  

가 되는거  
이렇게 해서 pad의 어텐션을 지울 수 있음

In [ ]:
src = torch.tensor([[19, 2, 3, 4, 1,1,1,1],  [19, 2, 5, 6, 7,8,9,11]])
src_pad_idx = 1

src_mask = (src != src_pad_idx).unsqueeze(1).unsqueeze(2)
src_mask.shape, src_mask

(torch.Size([2, 1, 1, 8]),
 tensor([[[[ True,  True,  True,  True, False, False, False, False]]],
 
 
         [[[ True,  True,  True,  True,  True,  True,  True,  True]]]]))

In [ ]:
class PositionwiseFeedforwardLayer(nn.Module):
  def __init__(self, hidden_dim, pf_dim, dropout_ratio):
    super().__init__()

    self.fc_1 = nn.Linear(hidden_dim, pf_dim)
    self.fc_2 = nn.Linear(pf_dim, hidden_dim)

    self.dropout = nn.Dropout(dropout_ratio)

  def forward(self, x):

    # x: [batch_size, seq_len, hidden_dim]

    x = self.dropout(torch.relu(self.fc_1(x)))

    # x: [batch_size, seq_len, pf_dim]

    x = self.fc_2(x)

    # x: [batch_size, seq_len, hidden_dim]

    return x

이제 Encoder 레이어를 구현하자.

하나의 인코더 레이어를 정의하고 나면, 몇 겁을 쌓으려하든, 그 이후부터는 그냥 이 레이어를 쌓기만 하면 된다. (input output의 shape가 같기 때문)



In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
    super().__init__()

    self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
    self.ff_layer_norm = nn.LayerNorm(hidden_dim)
    self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
    self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
    self.dropout = nn.Dropout(dropout_ratio)

    # 하나의 임베딩이 복제되어 Query, Key, Value로 입력되는 방식
  def forward(self, src, src_mask):

    # src: [batch_size, src_len, hidden_dim]
    # src_mask: [batch_size, src_len]

    # self attention
    # 필요한 경우 마스크(mask) 행렬을 이용하여 어텐션(attention)할 단어를 조절 가능
    _src, _ = self.self_attention(src, src, src, src_mask)

    # dropout, residual connection and layer norm
    src = self.self_attn_layer_norm(src + self.dropout(_src))

    # src: [batch_size, src_len, hidden_dim]

    # position-wise feedforward
    _src = self.positionwise_feedforward(src)

    # dropout, residual and layer norm
    src = self.ff_layer_norm(src + self.dropout(_src))

    # src: [batch_size, src_len, hidden_dim]

    return src

In [ ]:
class Encoder(nn.Module):
  def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
    super().__init__()

    self.device = device

    self.tok_embedding = nn.Embedding(input_dim, hidden_dim)
    self.pos_embedding = nn.Embedding(max_length, hidden_dim)

    self.layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

    self.dropout = nn.Dropout(dropout_ratio)

    self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

  def forward(self, src, src_mask):
    

    # src: [batch_size, src_len]
    # src_mask: [batch_size, src_len]

    batch_size = src.shape[0]
    src_len = src.shape[1]

    pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

    # pos: [batch_size, src_len]

    # 소스 문장의 임베딩과 위치 임베딩을 더한 것을 사용
    src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))

    # src: [batch_size, src_len, hidden_dim]

    # 모든 인코더 레이어를 차례대로 거치면서 순전파(forward) 수행
    for layer in self.layers:
        src = layer(src, src_mask)

    # src: [batch_size, src_len, hidden_dim]

    return src # 마지막 레이어의 출력을 반환

## Decoder part

Decoder에서 첫번째의 mask 파트는 쿼리 인코딩할때, 각각 쿼리마다 자신의 다음 단어를 참고 못하도록 해서 다음 attention(key와 value가 Encoder part의 output인 attention 부분)을 위한 쿼리 임베딩을 하는 것.
- 예를들어 첫 단어의 경우는 아무것도 참고 못하고 자기 자신의 query1*key1 = attention 1이 그냥 값이 1인것 -> 즉, 그냥 value 1 값 자체를 가져다 사용하겠다는 의미

In [ ]:
class DecoderLayer(nn.Module):
  def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
    super().__init__()

    self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
    self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)
    self.ff_layer_norm = nn.LayerNorm(hidden_dim)
    self.self_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device) # -> encoder_attention이 이거랑 같은건데 가중치를 다르게해서 따로 학습하려고 따로따로 선언한거
    self.encoder_attention = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
    self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
    self.dropout = nn.Dropout(dropout_ratio)

    # 인코더의 출력 값(enc_src)을 어텐션(attention)하는 구조
  def forward(self, trg, enc_src, trg_mask, src_mask):

    # trg: [batch_size, trg_len, hidden_dim]
    # enc_src: [batch_size, src_len, hidden_dim]
    # trg_mask: [batch_size, trg_len]
    # src_mask: [batch_size, src_len]

    # self attention
    # 자기 자신에 대하여 어텐션(attention)
    _trg, _ = self.self_attention(trg, trg, trg, trg_mask)

    # dropout, residual connection and layer norm
    trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

    # trg: [batch_size, trg_len, hidden_dim]

    # encoder attention
    # 디코더의 쿼리(Query)를 이용해 인코더를 어텐션(attention)
    _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)

    # dropout, residual connection and layer norm
    trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))

    # trg: [batch_size, trg_len, hidden_dim]

    # positionwise feedforward
    _trg = self.positionwise_feedforward(trg)

    # dropout, residual and layer norm
    trg = self.ff_layer_norm(trg + self.dropout(_trg))

    # trg: [batch_size, trg_len, hidden_dim]
    # attention: [batch_size, n_heads, trg_len, src_len]

    return trg, attention

In [ ]:
class Decoder(nn.Module):
  def __init__(self, output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length=100):
    super().__init__()

    self.device = device

    self.tok_embedding = nn.Embedding(output_dim, hidden_dim)
    self.pos_embedding = nn.Embedding(max_length, hidden_dim)

    self.layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

    self.fc_out = nn.Linear(hidden_dim, output_dim)

    self.dropout = nn.Dropout(dropout_ratio)

    self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

  def forward(self, trg, enc_src, trg_mask, src_mask):

    # trg: [batch_size, trg_len]
    # enc_src: [batch_size, src_len, hidden_dim]
    # trg_mask: [batch_size, trg_len]
    # src_mask: [batch_size, src_len]

    batch_size = trg.shape[0]
    trg_len = trg.shape[1]

    pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

    # pos: [batch_size, trg_len]

    trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))

    # trg: [batch_size, trg_len, hidden_dim]

    for layer in self.layers:
        # 소스 마스크와 타겟 마스크 모두 사용
        trg, attention = layer(trg, enc_src, trg_mask, src_mask)

    # trg: [batch_size, trg_len, hidden_dim]
    # attention: [batch_size, n_heads, trg_len, src_len]

    output = self.fc_out(trg)

    # output: [batch_size, trg_len, output_dim]

    return output, attention

In [ ]:
class Transformer(nn.Module):
  def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
    super().__init__()

    self.encoder = encoder
    self.decoder = decoder
    self.src_pad_idx = src_pad_idx
    self.trg_pad_idx = trg_pad_idx
    self.device = device

    # 소스 문장의 <pad> 토큰에 대하여 마스크(mask) 값을 0으로 설정
  def make_src_mask(self, src):

    # src: [batch_size, src_len]

    src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

    # src_mask: [batch_size, 1, 1, src_len]

    return src_mask

    # 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록(이전 단어만 보도록) 만들기 위해 마스크를 사용
  def make_trg_mask(self, trg):

    # trg: [batch_size, trg_len]

    """ (마스크 예시)
    1 0 0 0 0
    1 1 0 0 0
    1 1 1 0 0
    1 1 1 0 0
    1 1 1 0 0
    """
    trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)

    # trg_pad_mask: [batch_size, 1, 1, trg_len]

    trg_len = trg.shape[1]

    """ (마스크 예시)
    1 0 0 0 0
    1 1 0 0 0
    1 1 1 0 0
    1 1 1 1 0
    1 1 1 1 1
    """
    trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()

    # trg_sub_mask: [trg_len, trg_len]

    trg_mask = trg_pad_mask & trg_sub_mask

    # trg_mask: [batch_size, 1, trg_len, trg_len]

    return trg_mask

  def forward(self, src, trg):

    # src: [batch_size, src_len]
    # trg: [batch_size, trg_len]

    src_mask = self.make_src_mask(src)
    trg_mask = self.make_trg_mask(trg)

    # src_mask: [batch_size, 1, 1, src_len]
    # trg_mask: [batch_size, 1, trg_len, trg_len]

    enc_src = self.encoder(src, src_mask)

    # enc_src: [batch_size, src_len, hidden_dim]

    output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

    # output: [batch_size, trg_len, output_dim]
    # attention: [batch_size, n_heads, trg_len, src_len]

    return output, attention

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HIDDEN_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

In [ ]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

# 인코더(encoder)와 디코더(decoder) 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

# Transformer 객체 선언
model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,038,341 trainable parameters


In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

In [ ]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 뒷 부분의 패딩(padding)에 대해서는 값 무시
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
# 모델 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0

    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        # 출력 단어의 마지막 인덱스(<eos>)는 제외
        # 입력을 할 때는 <sos>부터 시작하도록 처리
        output, _ = model(src, trg[:,:-1])

        # output: [배치 크기, trg_len - 1, output_dim]
        # trg: [배치 크기, trg_len]

        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        # 출력 단어의 인덱스 0(<sos>)은 제외
        trg = trg[:,1:].contiguous().view(-1)

        # output: [배치 크기 * trg_len - 1, output_dim]
        # trg: [배치 크기 * trg len - 1]

        # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward() # 기울기(gradient) 계산

        # 기울기(gradient) clipping 진행
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 파라미터 업데이트
        optimizer.step()

        # 전체 손실 값 계산
        epoch_loss += loss.item()

        break

    return epoch_loss / len(iterator)

In [ ]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
  model.eval() # 평가 모드
  epoch_loss = 0

  with torch.no_grad():
      # 전체 평가 데이터를 확인하며
    for i, batch in enumerate(iterator):
      src = batch.src
      trg = batch.trg

      # 출력 단어의 마지막 인덱스(<eos>)는 제외
      # 입력을 할 때는 <sos>부터 시작하도록 처리
      output, _ = model(src, trg[:,:-1])

      # output: [배치 크기, trg_len - 1, output_dim]
      # trg: [배치 크기, trg_len]

      output_dim = output.shape[-1]

      output = output.contiguous().view(-1, output_dim)
      # 출력 단어의 인덱스 0(<sos>)은 제외
      trg = trg[:,1:].contiguous().view(-1)

      # output: [배치 크기 * trg_len - 1, output_dim]
      # trg: [배치 크기 * trg len - 1]

      # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
      loss = criterion(output, trg)

      # 전체 손실 값 계산
      epoch_loss += loss.item()
      break

  return epoch_loss / len(iterator)

In [ ]:
import math
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import time
import math
import random

N_EPOCHS = 1
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer_german_to_english.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
           -3.1856e-01,  1.0287e+00]]],


        [[[ 5.3388e-01, -1.0000e+10, -1.0000e+10,  ..., -1.0000e+10,
           -1.0000e+10, -1.0000e+10],
          [-5.1516e-01, -8.0987e-01, -1.0000e+10,  ..., -1.0000e+10,
           -1.0000e+10, -1.0000e+10],
          [ 6.4489e-01,  5.0683e-02,  6.1418e-01,  ..., -1.0000e+10,
           -1.0000e+10, -1.0000e+10],
          ...,
          [-3.6920e-01, -3.9639e-01, -4.0471e-01,  ..., -3.5988e-02,
           -1.0000e+10, -1.0000e+10],
          [-2.5045e-01, -1.4873e-02,  5.4099e-01,  ...,  3.8451e-02,
            7.2532e-01, -1.0000e+10],
          [-5.1750e-01,  1.9730e-01,  4.3671e-01,  ...,  1.6139e-02,
            4.2520e-01,  1.4768e+00]],

         [[-7.2654e-01, -1.0000e+10, -1.0000e+10,  ..., -1.0000e+10,
           -1.0000e+10, -1.0000e+10],
          [-1.7387e+00, -4.8419e-01, -1.0000e+10,  ..., -1.0000e+10,
           -1.0000e+10, -1.0000e+10],
          [-1.1190e+00, -5.5187e-01, -4.9256e-01,